In [1]:
# MNIST ANN Pytorch practice_0.52

In [24]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sys # to be used in showing figure in Tensorboard
# visualize data with matplot
"""
import matplotlib.pyplot as plt 
"""

# visualize data with TensorBoard localhost:6006

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/mnist")


#device configuration using cpu only
"""
device = torch.device('cpu')
"""

# use gpu: 
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
"""

"\ndevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\n"

In [25]:
# define hyperparameters
input_size = 784 # image pixel 28 x 28
hidden_size = 100  # the higher the more accurate
output_size = 10  # number of classes 0 to 9
epochs = 10
batch_size = 100
learning_rate = 0.001

# import MNIST dataset
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, 
                                           transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, 
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# to view data

examples = iter(train_loader)
samples, labels = next(examples)
print(samples.shape, labels.shape)

"""
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(samples[i][0], cmap='gray')
"""

# view data on Tensorboard localhost:6006, but get blank page

img_grid = torchvision.utils.make_grid(samples)
writer.add_image('mnist_images', img_grid)
writer.close 
# sys.exit() 


torch.Size([100, 1, 28, 28]) torch.Size([100])


<bound method SummaryWriter.close of <torch.utils.tensorboard.writer.SummaryWriter object at 0x7fb718b64040>>

In [26]:
class Ann(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Ann, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) # hidden layer from input
        self.relu = nn.ReLU() # activation function
        self.l2 = nn.Linear(hidden_size, output_size) # hidden layer befor output
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out



In [27]:
# create model
model = Ann(input_size, hidden_size, output_size)

In [28]:
# loss and optimizer
criterion = nn.CrossEntropyLoss() # use cross entropy loss as loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # use Adam as optimizer

#visualize with tensorboard

writer.add_graph(model, samples.reshape(-1, 28*28))
writer.close()
sys.exit

<function sys.exit(status=None, /)>

In [29]:
# training loop
number_steps = len(train_loader)

# for tensorboard views (TBV)

running_loss = 0.0  # for TBV
running_correct = 0 # for TBV


for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28) # reshape images's tensor 1, 28, 28 to 784
        labels = labels
        
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #backward pass
        optimizer.zero_grad() # empty the value in the gradian decent
        loss.backward() # backward propagation
        optimizer.step() # update step

        ### for TBV ###
        running_loss == loss.item() 
        _, predicted = torch.max(outputs, 1) 
        running_correct += (predicted == labels).sum().item() 
        ###############
        
        # print information of each epoch
        if (i+1) % 600 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{number_steps}], Loss: {loss.item():.4f}')
        
        ### for TBV ###
        writer.add_scalar('training loss', running_loss /100, epoch * number_steps +i)
        running_accuracy = running_correct /100 / predicted.size(0)
        writer.add_scalar('accuracy', running_accuracy, epoch * number_steps + i)
        running_correct = 0
        running_loss = 0.0
        #################

# result shows training lost decreasing to around 0.03
            

Epoch [1/10], Step [600/600], Loss: 0.2777
Epoch [2/10], Step [600/600], Loss: 0.2507
Epoch [3/10], Step [600/600], Loss: 0.1399
Epoch [4/10], Step [600/600], Loss: 0.1418
Epoch [5/10], Step [600/600], Loss: 0.0721
Epoch [6/10], Step [600/600], Loss: 0.0484
Epoch [7/10], Step [600/600], Loss: 0.0522
Epoch [8/10], Step [600/600], Loss: 0.0174
Epoch [9/10], Step [600/600], Loss: 0.0771
Epoch [10/10], Step [600/600], Loss: 0.0335


In [ ]:
#test model

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for impages, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        output = model(images)
        # max returns (value, index)
        _, prediction = torch.max(outputs, 1) 
        n_samples += labels.shape[0]
        n_correct += (prediction == labels).sum().item()
    
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

    # Accuracy rate of the network on the 10000 test images: 51.825 % with learning rate of 0.001
    # Accuracy rate improved to  around 53 % when lower the learning rate at 0.01
    

